In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
import logging

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))
sys.path.append(os.path.expanduser('..'))

from generate_common import cache_load
from spinorama.compute_cea2034 import sound_power
from spinorama.load import graph_melt
from spinorama.graph import graph_params_default, contour_params_default, isoband_params_default

import datas.metadata as metadata
df = cache_load(smoke_test=False)
print(df.keys())

In [ ]:
# dfu = df['KEF LS50']['ASR']['asr']['SPL Horizontal_unmelted']
# dfu = df['JBL 708P']['ASR']['asr']['SPL Horizontal_unmelted']
# dfu = df['Genelec 8341A']['ASR']['asr-vertical']['SPL Vertical_unmelted']
# dfu = df['Genelec 8341A']['ASR']['asr-vertical']['SPL Horizontal_unmelted']
# dfu = df['Genelec 8030C']['ASR']['asr']['SPL Horizontal_unmelted']
# dfu = df['Wharfedale Diamond 220']['ASR']['asr']['SPL Vertical_unmelted']
# dfu = df['Revel C208']['ASR']['asr']['SPL Horizontal_unmelted']
# dfu = df['KEF LS50']['ASR']['asr-horizontal']['SPL Horizontal_unmelted']
# dfu = df['KRK Systems Classic 5']['ASR']['asr']['SPL Vertical_unmelted']
# dfu = df['Verdant Audio Bambusa MG 1']['ASR']['asr']['SPL Vertical_unmelted']
# dfu = df['Neumann KH 80']['ASR']['asr-v3-20200711']['SPL Vertical_unmelted']
dfu = df['Neumann KH 80']['ASR']['asr-v3-20200711']['SPL Horizontal_unmelted']


In [ ]:
from spinorama.graph_contour import compute_contour, compute_contour_smoothed
# af, am, az = compute_contour_smoothed(dfu)
af, am, az = compute_contour(dfu)
len(am.T[0])

In [ ]:
# more interesting to look at -3/0 range                                                                                            
speaker_scale = contour_params_default["contour_scale"]
colormap = "veridis"

freq = af.ravel()
angle = am.ravel()
db = az.ravel()

source = pd.DataFrame({"Freq": freq, "Angle": angle, "dB": db})

chart = alt.Chart(source
).mark_rect(
).transform_filter("datum.Freq>400"
).encode(
    alt.X("Freq:O", axis=None),
    alt.Y("Angle:O", title="Angle (deg.)", axis=None, sort=None),
    alt.Color("dB:Q", scale=alt.Scale(scheme="spectral", domain=speaker_scale, nice=True))
).properties(width=800, height=600)

Find angle such that we model the directivity left/right or up/down

In [ ]:
def func(x, sign):
    xp1 = int(x)
        # print(am.T[0][xp1])
    xp2 = xp1+sign
    zp1 = az[xp1][110:180]
    zp2 = az[xp2][110:180]
    # linear interpolation
    zp  = zp1+(x-xp1)*(zp2-zp1)
    # normˆ2 (z-(-6dB))
    return np.linalg.norm(zp+6)

eval_count = 180
space_p = np.linspace(int(len(am.T[0])/2), 1, eval_count)
eval_p = [func(x, +1) for x in space_p]
min_p = np.min(eval_p)*1.1
pos_g = [i for i, v in enumerate(eval_p) if v < min_p]
pos_p = pos_g[-1]
angle_p = pos_p*180/eval_count
chart_p = alt.Chart(pd.DataFrame({'x': np.linspace(0, 180, eval_count), 'y': eval_p})).mark_line().encode(x='x:Q', y='y:Q')

space_m = np.linspace(int(len(am.T[0])/2), len(am.T[0])-2, eval_count)
eval_m = [func(x, 1) for x in space_m]
min_m = np.min(eval_m)*1.05
pos_g = [i for i, v in enumerate(eval_m) if v < min_m]
pos_m = pos_g[-1]
print(pos_g)
angle_m = -pos_m*180/eval_count
chart_m = alt.Chart(pd.DataFrame({'x': np.linspace(0, -180, eval_count), 'y': eval_m})).mark_line().encode(x='x:Q', y='y:Q')


#print('pos_p {}'.format(pos_p))
#print('space_p int min {} int max {}'.format(int(np.min(space_p)), int(np.max(space_p))))
#print('space_p[pos_p] = {}'.format(space_p[pos_p]/(space_p[0]-space_p[-1])))
print(angle_p, angle_m, (angle_p-angle_m)/2)
chart_m|chart_p

In [ ]:
f_min = 400
x_axis = alt.X('Freq:Q', axis=None, scale=alt.Scale(type="log", domain=[f_min, 20000], nice=False))
y_axis = y=alt.Y('Angle:Q', scale=alt.Scale(domain=[-180, 180], nice=False))
color = alt.value('white')
size=alt.value(3)

line_source = pd.DataFrame({"Freq": [f_min, 20000], "Angle": [0,0], "dB": [0,0]})
line_source_p = pd.DataFrame({"Freq": [f_min, 20000], "Angle": [angle_p, angle_p], "dB": [+6,+6]})
line_source_m = pd.DataFrame({"Freq": [f_min, 20000], "Angle": [angle_m, angle_m], "dB": [-6,-6]})

line = alt.Chart(line_source).mark_line().encode(x=x_axis, y=y_axis, color=color, size=size)
line_p = alt.Chart(line_source_p).mark_line().encode(x=x_axis, y=y_axis, color=color, size=size)
line_m = alt.Chart(line_source_m).mark_line().encode(x=x_axis, y=y_axis, color=color, size=size)

In [ ]:
(chart+line+line_p+line_m).properties(width=800, height=600)